# Monte-Carlo Pricing with the Heston Model

Authors: Artemy Sazonov, Danil Legenkiy, and Kirill Korban

Supervisors: Mikhail Zhitlukhin, Charles-Henri Roubinet

Objectives:
1. Implement the most common discretization schemes for the Heston model.
2. Compare the performance of the schemes and compare them with the closed-form solution for the European call option;
3. Test the exotic options pricing with the Heston model.

In [2]:
from hestonmc import *
from derivatives import *
from vol.vol import Heston

import numpy as np

In [3]:
heston_params = HestonParameters(kappa = 0.5, gamma = 1, rho = -0.9, vbar = 0.04, v0 = 0.04)

state         = MarketState(stock_price = 100, interest_rate = 0.)
model         = Heston(state.stock_price, heston_params.v0, heston_params.kappa, heston_params.vbar, heston_params.gamma, heston_params.rho, state.interest_rate)

r_x           = np.load(r"Data/truncated_gaussian/r_x start=1e-07 stop=100 N=4999998 dt=2e-05.npy")
f_nu_y        = np.load(r"Data/truncated_gaussian/f_nu_y start=1e-07 stop=100 N=4999998 dt=2e-05.npy")
f_sigma_y     = np.load(r"Data/truncated_gaussian/f_sigma_y start=1e-07 stop=100 N=4999998 dt=2e-05.npy")

kwargs        = {
                    'x_grid' : r_x, 
                    'f_nu_grid' : f_nu_y, 
                    'f_sigma_grid' : f_sigma_y 
                }

## Comparison of the discretization schemes and the closed-form solution

### At-the-money European call option

In [4]:
strike = state.stock_price
T = 1
ec_payoff = european_call_payoff(T, strike, state.interest_rate)

common_mc_params = {"absolute_error": 1e-2, "state": state, "heston_params": heston_params, "payoff": ec_payoff, "T": T}

#### Closed-form price

In [5]:
model.call_price(T, strike)

4.40338420430231

#### Euler scheme

In [14]:
mc_price(N_T = 40, simulate = simulate_heston_euler, batch_size = 10_000, verbose = True, **common_mc_params)

Number of simulate calls:   80
MAX_ITER:                   100000
Number of paths:            3200000
Absolute error:             0.01
Length of the conf intl:    0.009950413572929773
Confidence level:           0.05



4.7906063174725215

In [18]:
mc_price(N_T = 200, simulate = simulate_heston_euler, batch_size = 10_000, verbose = True, **common_mc_params)

Number of simulate calls:   68
MAX_ITER:                   100000
Number of paths:            2720000
Absolute error:             0.01
Length of the conf intl:    0.009928552532378562
Confidence level:           0.05



4.458799582002607

In [30]:
mc_price(N_T = 40, simulate = simulate_heston_andersen_qe, batch_size = 10_000, verbose = True, **common_mc_params)

Number of simulate calls:   63
MAX_ITER:                   100000
Number of paths:            2520000
Absolute error:             0.01
Length of the conf intl:    0.009969524260968783
Confidence level:           0.05



4.390740187286735

As we can see, the Euler scheme requires a lot more points to achieve the same accuracy as the Andersen schemes.

#### Andersen schemes

In [28]:
mc_price(N_T = 40, simulate = simulate_heston_andersen_tg, batch_size = 10_000, verbose = True, **common_mc_params, **kwargs)

Number of simulate calls:   64
MAX_ITER:                   100000
Number of paths:            2560000
Absolute error:             0.01
Length of the conf intl:    0.009996494292748669
Confidence level:           0.05



4.395463982322609

#### Speed comparison

In [19]:
%timeit mc_price(N_T = 200, simulate = simulate_heston_euler, batch_size = 10_000, **common_mc_params)

1.9 s ± 14.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
%timeit mc_price(N_T = 40, simulate = simulate_heston_andersen_qe, batch_size = 10_000, **common_mc_params)

578 ms ± 4.23 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%timeit mc_price(N_T = 40, simulate = simulate_heston_andersen_tg, batch_size = 10_000, **common_mc_params, **kwargs)

511 ms ± 7.46 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Exotic options pricing

Here we provide a basic example of the Asian Call option pricing (arithmetic mean) with and without the control variate.

In [18]:
asian_call = asian_call_AM_payoff(T, state.stock_price)
asian_mc_params = {"N_T": 50, "simulate": simulate_heston_andersen_qe, "batch_size": 20_000, "absolute_error": 1e-3, "state": state, "heston_params": heston_params, "payoff": asian_call, "T": T}

In [19]:
mc_price(**asian_mc_params, verbose = True)

Number of simulate calls:   1417
MAX_ITER:                   100000
Number of paths:            113360000
Absolute error:             0.001
Length of the conf intl:    0.0009996823381729775
Confidence level:           0.05



3.095672658360093

In [20]:
mc_price(**asian_mc_params, control_variate_payoff=ec_payoff, mu = model.call_price(T, strike), verbose = True)

Control variate payoff:     european_call
Control variate iterations: 1000
Number of simulate calls:   445
MAX_ITER:                   100000
Number of paths:            35600000
Absolute error:             0.001
Length of the conf intl:    0.0009991528466140344
Confidence level:           0.05



3.1014962948789946

In [21]:
%timeit -r3 mc_price(**asian_mc_params)

45.5 s ± 144 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [22]:
%timeit -r3 mc_price(**asian_mc_params, control_variate_payoff=ec_payoff, mu = model.call_price(T, strike))

14.4 s ± 57.6 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
